[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Aleph-Alpha/examples/blob/main/boilerplate/08_gradio_app.ipynb)

# Notebook 8: A simple gradio app for extracting text from emails
Finally, in this notebook we will try to create a demo application with a small interface

This will help to assess how good a task can be completed.

In [ ]:
!pip install aleph_alpha_client gradio

from typing import Sequence
from aleph_alpha_client import Client, Prompt, CompletionRequest
import gradio as gr

import math
import os

In [ ]:
client = Client(token="AA_TOKEN")

## Creating a small data extraction demo
We will create a small demo that extracts the most relevant information from emails. We will visualize the results in a small interface using gradio. 

The data we will extract is:
- The sender
- The receiver
- The intent of the email

In [ ]:
def extract_sender(text: str):
    prompt = """This system extracts the sender of an email from the text.
###
Text: Hi,
This is Lara Gruppa from System IT.
I wanted to check if you got our email.
Please respond if possible.
Best,
Lara Gruppe
Sender: Lara Gruppa
###
Text: Dear Mr Daniels,
Really pleased to hear that you presentation went well.
I hope we can have more such cooperations in the future.
Kind Regards,
Peter Thomas
Sender: Peter Thomas
###
From: Tobias Denzen<Tobias.Denzen@world.de> 
Sent: Friday, November 25, 2022 8:04 AM
To: Dieter, Knüppelhorn <dieter.knue@other.com>
Subject: AW: We want to get the Use Case going
Good morning Dieter,
and thanks to all for the time spent in preparation.
We are really excited to get the use case going with the special administration.
Best regards.
Tobias
Sender: Tobias Denzen
###
Text: Hello Sam Tomson,
I would like to let you know, that you have been selected for jury duty.
Please register yourself until coming sunday.
Sender: No sender
###
Hi Patrick Loranz,
Franz Dimer here from the local food store.
Looks like you forgot your wallet here.
You can pick it up any time.
Best
Sender: Franz Dimer
###
Text: {}
Sender:"""
    
    request = CompletionRequest(prompt=Prompt.from_text(prompt.format(text)), stop_sequences=["###"], maximum_tokens=20)
    completion = client.complete(request, model="luminous-extended").completions[0].completion
    sender = completion
    return sender

def extract_receiver(text: str):
    prompt = """This system extracts the Recipient of an email from the text.
###
Text: Hi,
This is Lara Gruppa from System IT.
I wanted to check if you got our email.
Please respond if possible.
Best,
Lara Gruppe
Recipient: No Recipient
###
Text: Dear Mr. Daniels,
Really pleased to hear that you presentation went well.
I hope we can have more such cooperations in the future.
Kind Regards,
Peter Thomas
Recipient: Mr. Daniels
###
From: Tobias Denzen<Tobias.Denzen@world.de> 
Sent: Friday, November 25, 2022 8:04 AM
To: Dieter Knüppelhorn <dieter.knue@other.com>
Subject: AW: We want to get the Use Case going
Good morning Dieter,
and thanks to all for the time spent in preparation.
We are really excited to get the use case going with the special administration.
Best regards.
Tobias
Recipient: Dieter Knüppelhorn
###
Hi,
Thanks for the great training on Friday. 
I really learned a lot.
Best,
Simon Esser
Recipient: No Recipient
###
Text: Hello Sam Tomson,
I would like to let you know, that you have been selected for jury duty.
Please register yourself until coming sunday.
Recipient: Sam Tomson
###
Hi,
Franz Dimer here from the local food store.
You must be Patrick Loranz.
Looks like you forgot your wallet here.
You can pick it up any time.
Best
Recipient: Patrick Loranz
###
Text: {}
Recipient:"""
    
    request = CompletionRequest(prompt=Prompt.from_text(prompt.format(text)), stop_sequences=["###"], maximum_tokens=20)
    completion = client.complete(request, model="luminous-extended").completions[0].completion
    receiver = completion
    return receiver

# TODO : write a prompt that extracts the intent from the email.
def extract_intent(text: str):
    prompt = """This system extracts the Intent of an email from the text.
###
Text: Hi,
This is Lara Gruppa from System IT.
I wanted to check if you got our email.
Please respond if possible.
Best,
Lara Gruppe
Intent: Check-in on email
###
Text: Dear Mr. Daniels,
Really pleased to hear that you presentation went well.
I hope we can have more such cooperations in the future.
Kind Regards,
Peter Thomas
Intent: Congratulations on Presentation
###
From: Tobias Denzen<Tobias.Denzen@world.de> 
Sent: Friday, November 25, 2022 8:04 AM
To: Dieter Knüppelhorn <dieter.knue@other.com>
Subject: AW: We want to get the Use Case going
Good morning Dieter,
and thanks to all for the time spent in preparation.
We are really excited to get the use case going with the special administration.
Best regards.
Tobias
Intent: Use Case with the special administration
###
Text: Hello Sam Tomson,
I would like to let you know, that you have been selected for jury duty.
Please register yourself until coming sunday.
Intent: Registering for Jury Duty
###
Hi,
Franz Dimer here from the local food store.
You must be Patrick Loranz.
Looks like you forgot your wallet here.
You can pick it up any time.
Best
Intent: Picking up wallet
###
Text: {}
Intent:"""
    
    request = CompletionRequest(prompt=Prompt.from_text(prompt.format(text)), stop_sequences=["###"], maximum_tokens=20)
    completion = client.complete(request, model="luminous-extended").completions[0].completion
    intent = completion
    return intent

In [ ]:
# Small helper fuction that runs the above functions and returns the results
def run(text: str):
    sender = extract_sender(text)
    receiver = extract_receiver(text)
    intent = extract_intent(text)
    return sender, receiver, intent

## Visualizing the results
AI can be hard to understand. To make it easier for us to understand the results we will visualize them using gradio. Gradio is a library that allows you to create a small interface for your AI model. You can find more information about gradio [here](https://gradio.app/).

In [ ]:
# Gradio Interface that accepts a text and returns the sender, receiver and intent
interface = gr.Interface(
    fn=run,
    inputs=gr.components.Textbox(lines=10, placeholder="Enter your email here"),
    outputs=[
        gr.components.Textbox(label="Sender"),
        gr.components.Textbox(label="Receiver"),
        gr.components.Textbox(label="Intent"),
    ],
    title="Email Intent Extraction",
    description="This model extracts the sender, receiver and intent from an email."
)
interface.launch()